In [1]:
import re
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import itertools
from math import inf

# My Solution to Exercise 1

In [27]:
def obtain_graph(source = 'graph.txt'):
    with open(source, 'r') as f:
        data = f.read().splitlines()
    
    nodes = {}
    G_nodes = {}
    for idx, line in enumerate(data):
        node = re.search(r'node%s' % int(idx), line).group()
        vals = re.findall(r'node\d+\s\d+\.{,1}\d+', line)
        nodes[node] = [(entry.split(' ')[0], float(entry.split(' ')[1])) for entry in vals]
        G_nodes[node] = {n: {'weight': w} for n,w in nodes[node]}
    
    return nx.DiGraph(G_nodes)
        
def draw_graph(graph, lbl=True, arr=False):
    nx.draw(graph, with_labels=lbl, arrows=arr)
    plt.show()

def get_shortest_path(graph, start, end):
    paths = {}
    distances = {}
    nodes = graph.nodes()
    edges = nx.to_dict_of_dicts(graph)

    for node in nodes:
        distances[node] = inf

    distances[start] = 0

    for i in range(len(nodes) - 1):
        updates = 0    
        for unode in nodes:
            for vnode, weight in edges[unode].items():
                temp_dist = weight['weight'] + distances[unode]
                if temp_dist < distances[vnode]:
                    distances[vnode] = temp_dist
                    paths[vnode] = unode
                    updates += 1
        if updates == 0:
            break                    
                
    current = end
    path = [current]

    while current != start:
        current = paths[current]
        path.append(current)
    
    return distances[end], path[::-1]

In [28]:
txt = 'graph.txt'
G = obtain_graph(txt)

In [32]:
cost, path = get_shortest_path(G, 'node0', 'node99')

In [36]:
path, cost

(['node0',
  'node8',
  'node11',
  'node18',
  'node23',
  'node33',
  'node41',
  'node53',
  'node56',
  'node57',
  'node60',
  'node67',
  'node70',
  'node73',
  'node76',
  'node85',
  'node87',
  'node88',
  'node93',
  'node94',
  'node96',
  'node97',
  'node98',
  'node99'],
 160.55000000000007)

# Text Solution to Exercise 1

In [34]:
def read_graph(in_file):
    """ Read in the graph from the data file.  The graph is stored
    as a dictionary, where the keys are the nodes, and the values
    are a list of pairs (d, c), where d is a node and c is a number.
    If (d, c) is in the list for node n, then d can be reached from
    n at cost c.
    """
    graph = {}
    infile = open(in_file)
    for line in infile:
        elements = line.split(',')
        node = elements.pop(0)
        graph[node] = []
        if node != 'node99':
            for element in elements:
                destination, cost = element.split()
                graph[node].append((destination, float(cost)))
    infile.close()
    return graph

def update_J(J, graph):
    "The Bellman operator."
    next_J = {}
    for node in graph:
        if node == 'node99':
            next_J[node] = 0
        else:
            next_J[node] = min(cost + J[dest] for dest, cost in graph[node])
    return next_J

def print_best_path(J, graph):
    """ Given a cost-to-go function, computes the best path.  At each node n,
    the function prints the current location, looks at all nodes that can be
    reached from n, and moves to the node m which minimizes c + J[m], where c
    is the cost of moving to m.
    """
    sum_costs = 0
    current_location = 'node0'
    while current_location != 'node99':
        print(current_location)
        running_min = 1e100  # Any big number
        for destination, cost in graph[current_location]:
            cost_of_path = cost + J[destination]
            if cost_of_path < running_min:
                running_min = cost_of_path
                minimizer_cost = cost
                minimizer_dest = destination
        current_location = minimizer_dest
        sum_costs += minimizer_cost

    print('node99\n')
    print('Cost: ', sum_costs)

In [46]:
## Main loop

graph = read_graph('graph.txt')
M = 1e10
J = {}
for node in graph:
    J[node] = M
J['node99'] = 0

while True:
    next_J = update_J(J, graph)
    if next_J == J:
        break
    else:
        J = next_J

print_best_path(J, graph)

node0
node8
node11
node18
node23
node33
node41
node53
node56
node57
node60
node67
node70
node73
node76
node85
node87
node88
node93
node94
node96
node97
node98
node99

Cost:  160.55000000000007
